In [162]:
!pip install SBTi
!pip install datacompy

In [238]:
import pandas as pd
import datacompy

import SBTi
from SBTi.configs import ColumnsConfig
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.interfaces import ETimeFrames, EScope
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod

In [203]:
df_raw = pd.read_excel("data_provider_example.xlsx", sheet_name="sample").head(10)
df_raw

,company_name,company_id,isic,target_type,intensity_metric,ghg_s1s2,ghg_s3,scope,coverage_s1,coverage_s2,...,s1s2s3_wats,s1s2_tets,s3_tets,s1s2s3_tets,owned_emissions_s1s2_rots,s1s2_rots,owned_emissions_s3,s3_rots,owned_emissions_s1s2s3,s1s2s3_rots
0,Company A,10001.0,0.0,Absolute,0.0,10000.0,20000.0,S1+S2+S3,0.80,1.0,...,0.006027,0.000871,0.004593,0.002112,5.000000e+03,0.001014,1.000000e+04,0.004557,1.500000e+04,0.002324
1,Company B,10002.0,0.0,Absolute,0.0,5000.0,2500.0,S1+S2+S3,1.00,1.0,...,0.007808,0.000342,0.000341,0.000342,3.289474e+03,0.000525,1.644737e+03,0.000445,4.934211e+03,0.000495
2,Company C,10003.0,0.0,Absolute,0.0,1000000.0,750000.0,S1+S2+S3,1.00,1.0,...,0.014075,0.068489,0.150718,0.095881,1.000000e+05,0.015951,7.500000e+04,0.029907,1.750000e+05,0.021105
3,Company D,10004.0,0.0,Absolute,0.0,5000000.0,3000000.0,S1+S2+S3,0.65,1.0,...,0.023014,0.504656,0.409091,0.537493,6.250000e+06,1.469143,3.750000e+06,1.014690,1.000000e+07,1.478856
4,Company E,10005.0,0.0,Absolute,0.0,250000.0,50000.0,S1+S2+S3,0.66,1.0,...,0.228767,0.025083,0.006818,0.020036,8.333333e+04,0.019472,1.666667e+04,0.004510,1.000000e+05,0.014701
5,Company F,10006.0,0.0,Absolute,0.0,300000.0,500000.0,S1+S2+S3,1.00,1.0,...,0.312329,0.020547,0.068182,0.036473,6.000000e+04,0.009570,1.000000e+05,0.027058,1.600000e+05,0.016056
6,Company G,10007.0,0.0,Absolute,0.0,1000.0,500.0,S1+S2+S3,0.50,1.0,...,0.789041,0.000115,0.000068,0.000115,6.000000e+03,0.001612,3.000000e+03,0.000812,9.000000e+03,0.001521
7,Company H,10008.0,0.0,Absolute,0.0,75000.0,35000.0,S1+S2+S3,1.00,1.0,...,0.062466,0.005137,0.004773,0.005015,1.200000e+04,0.001914,5.600000e+03,0.001515,1.760000e+04,0.001766
8,Company I,10009.0,0.0,Absolute,0.0,10000000.0,4000000.0,S1+S2+S3,1.00,1.0,...,0.078082,0.684890,0.545455,0.638272,6.250000e+05,0.099692,2.500000e+05,0.067646,8.750000e+05,0.087807
9,Company J,10010.0,0.0,Absolute,0.0,4000.0,2000.0,S1+S2+S3,1.00,1.0,...,0.023425,0.000274,0.000273,0.000274,2.400000e+03,0.000383,1.200000e+03,0.000325,3.600000e+03,0.000361


In [251]:
provider = ExcelProvider("data_provider_example.xlsx")
portfolio = pd.read_excel("data_provider_example.xlsx", sheet_name="portfolio")
companies = SBTi.utils.dataframe_to_portfolio(portfolio)

time_frames = [ETimeFrames.MID]
scopes = [EScope.S1S2S3, EScope.S1S2, EScope.S3]
fallback_score = 1.5

temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    aggregation_method=PortfolioAggregationMethod.ROTS 
)

df_scores = temperature_score.calculate(data_providers=[provider], portfolio=companies)
df_scores_subset = df_scores[["company_id", "scope", "reduction_ambition", "annual_reduction_rate", "temperature_score"]]

df_scores_subset_s1s2 = df_scores_subset[df_scores_subset["scope"] == EScope.S1S2].\
	rename(columns={"reduction_ambition": "s1s2_reduction_ambition", \
        "annual_reduction_rate": "s1s2_lar", \
        "temperature_score": "s1s2_ts" \
	}).drop(columns=["scope"])
df_scores_subset_s3 = df_scores_subset[df_scores_subset["scope"] == EScope.S3].\
	rename(columns={"reduction_ambition": "s3_reduction_ambition", \
        "annual_reduction_rate": "s3_lar", \
        "temperature_score": "s3_ts" \
	}).drop(columns=["scope"])
df_scores_subset_s1s2s3 = df_scores_subset[df_scores_subset["scope"] == EScope.S1S2S3]. \
	rename(columns={"temperature_score" : "ghg_s1s2s3_ts", \
	}).drop(columns=["reduction_ambition", "annual_reduction_rate", "scope"])

df_scores_merge = df_scores_subset_s1s2.merge(df_scores_subset_s3).merge(df_scores_subset_s1s2s3).astype({"company_id": "int64"}) 

cols = ["company_id", "s1s2_lar", "s3_lar", "s1s2_ts", "s3_ts", "ghg_s1s2s3_ts"]

# Temperature Score Compare
compare = datacompy.Compare(df_scores_merge[cols], df_raw[cols], join_columns= "company_id")
print(compare.report())

# Temperature Score Visualization
df_scores_compare = df_scores_merge[cols].merge(df_raw[cols],left_on="company_id", right_on="company_id", suffixes=("_s", "_p"))
df_scores_compare


DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        6    10
1       df2        6    10

Column Summary
--------------

Number of columns in common: 6
Number of columns in df1 but not in df2: 0
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: company_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 10
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 10

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 6
Total number of values which compare unequal: 0

Columns with Unequal Values or Types
------------------------------------

          Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null Diff
1     co

,company_id,s1s2_lar_s,s3_lar_s,s1s2_ts_s,s3_ts_s,ghg_s1s2s3_ts_s,s1s2_lar_p,s3_lar_p,s1s2_ts_p,s3_ts_p,ghg_s1s2s3_ts_p
0,10001,0.0400,0.0250,1.45,1.92,1.76,0.0400,0.0250,1.45,1.92,1.76
1,10002,0.0500,0.0500,1.14,1.14,1.14,0.0500,0.0500,1.14,1.14,1.14
2,10003,0.0500,0.0325,1.14,1.68,1.37,0.0500,0.0325,1.14,1.68,1.37
3,10004,0.0325,0.0500,1.68,1.14,1.68,0.0325,0.0500,1.68,1.14,1.68
4,10005,0.0330,0.0500,1.67,1.14,1.67,0.0330,0.0500,1.67,1.14,1.67
5,10006,0.0500,0.0500,1.14,1.14,1.14,0.0500,0.0500,1.14,1.14,1.14
6,10007,0.0250,0.0500,1.92,1.14,1.92,0.0250,0.0500,1.92,1.14,1.92
7,10008,0.0500,0.0500,1.14,1.14,1.14,0.0500,0.0500,1.14,1.14,1.14
8,10009,0.0500,0.0500,1.14,1.14,1.14,0.0500,0.0500,1.14,1.14,1.14
9,10010,0.0500,0.0500,1.14,1.14,1.14,0.0500,0.0500,1.14,1.14,1.14


In [195]:
aggregated_scores = temperature_score.aggregate_scores(df_scores)

In [233]:
def create_contributions_df(contributions, name):
	df = pd.DataFrame(
            columns = ["company_id", "company_name", "temperature_score", "contribution_relative", "contribution"])

	for c in contributions:
		insert_loc = df.index.max()
		row = [c.company_id, c.company_name, c.temperature_score, c.contribution_relative, c.contribution] 

		if pd.isna(insert_loc):
			df.loc[0] = row
		else:
			df.loc[insert_loc + 1] = row
		
	return df \
		.astype({"company_id": "int64"}) \
		.rename(columns={"contribution": name}) \
		.sort_values(by=["company_id"])

def create_merged_contributions_df(agg_method_name):

	s1s2_name = "s1s2_" + agg_method_name
	df_s1s2_contributions = create_contributions_df(aggregated_scores.mid.S1S2.all.contributions, name=s1s2_name)

	s3_name = "s3_" + agg_method_name
	df_s3_contributions = create_contributions_df(aggregated_scores.mid.S3.all.contributions, name=s3_name)

	s1s2s3_name = "s1s2s3_" + agg_method_name
	df_s1s2s3_contributions = create_contributions_df(aggregated_scores.mid.S1S2S3.all.contributions, name=s1s2s3_name)

	df_contributions = df_s1s2_contributions \
			.merge(df_s3_contributions,left_on="company_id", right_on="company_id") \
			.merge(df_s1s2s3_contributions, left_on="company_id", right_on="company_id")

	cols = ["company_id", s1s2_name, s3_name, s1s2s3_name]

	return df_contributions[cols], cols

In [242]:
df_sbti_contributions, cols = create_merged_contributions_df("rots")

compare = datacompy.Compare(df_sbti_contributions, df_raw[cols], join_columns= "company_id")
print(compare.report())

df_contributions = df_sbti_contributions.merge(df_raw[cols], left_on="company_id", right_on="company_id", suffixes=("_s", "_p"))
df_contributions

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        4    10
1       df2        4    10

Column Summary
--------------

Number of columns in common: 4
Number of columns in df1 but not in df2: 0
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: company_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 10
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 10

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 4
Total number of values which compare unequal: 0

Columns with Unequal Values or Types
------------------------------------

       Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null Diff
0  company_

,company_id,s1s2_rots_s,s3_rots_s,s1s2s3_rots_s,s1s2_rots_p,s3_rots_p,s1s2s3_rots_p
0,10001,0.001014,0.004557,0.002324,0.001014,0.004557,0.002324
1,10002,0.000525,0.000445,0.000495,0.000525,0.000445,0.000495
2,10003,0.015951,0.029907,0.021105,0.015951,0.029907,0.021105
3,10004,1.469143,1.014690,1.478856,1.469143,1.014690,1.478856
4,10005,0.019472,0.004510,0.014701,0.019472,0.004510,0.014701
5,10006,0.009570,0.027058,0.016056,0.009570,0.027058,0.016056
6,10007,0.001612,0.000812,0.001521,0.001612,0.000812,0.001521
7,10008,0.001914,0.001515,0.001766,0.001914,0.001515,0.001766
8,10009,0.099692,0.067646,0.087807,0.099692,0.067646,0.087807
9,10010,0.000383,0.000325,0.000361,0.000383,0.000325,0.000361


In [161]:
df_scores_merge

,company_id,s1s2_reduction_ambition,s1s2_lar,s1s2_ts,s3_reduction_ambition,s3_lar,s3_ts,ghg_s1s2s3_ts
0,10001,0.560,0.0400,1.45,0.350,0.0250,1.92,1.76
1,10002,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
2,10003,0.700,0.0500,1.14,0.455,0.0325,1.68,1.37
3,10004,0.455,0.0325,1.68,0.700,0.0500,1.14,1.68
4,10005,0.462,0.0330,1.67,0.700,0.0500,1.14,1.67
5,10006,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
6,10007,0.350,0.0250,1.92,0.700,0.0500,1.14,1.92
7,10008,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
8,10009,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
9,10010,0.700,0.0500,1.14,0.700,0.0500,1.14,1.14
